In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn.linear_model import LassoCV, RidgeCV
import lightgbm as lgb
from sklearn import preprocessing

In [2]:
train_df = pd.read_csv("../../../data/non-kaggle/Jeju-traffic/train.csv")
test_df = pd.read_csv("../../../data/non-kaggle/Jeju-traffic/test.csv")
data_info = pd.read_csv("../../../data/non-kaggle/Jeju-traffic/data_info.csv")
sample_submission = pd.read_csv("../../../data/non-kaggle/Jeju-traffic/sample_submission.csv")

In [3]:
print(train_df.head())

              id  base_date day_of_week  base_hour  lane_count  road_rating  \
0  TRAIN_0000000   20220623           목         17           1          106   
1  TRAIN_0000001   20220728           목         21           2          103   
2  TRAIN_0000002   20211010           일          7           2          103   
3  TRAIN_0000003   20220311           금         13           2          107   
4  TRAIN_0000004   20211005           화          8           2          103   

   road_name  multi_linked  connect_code  maximum_speed_limit  ...  road_type  \
0  지방도1112호선             0             0                 60.0  ...          3   
1   일반국도11호선             0             0                 60.0  ...          0   
2   일반국도16호선             0             0                 80.0  ...          0   
3        태평로             0             0                 50.0  ...          0   
4   일반국도12호선             0             0                 80.0  ...          0   

   start_node_name  start_latitude  st

In [4]:
print(test_df.head())

            id  base_date day_of_week  base_hour  lane_count  road_rating  \
0  TEST_000000   20220825           목         17           3          107   
1  TEST_000001   20220809           화         12           2          103   
2  TEST_000002   20220805           금          2           1          103   
3  TEST_000003   20220818           목         23           3          103   
4  TEST_000004   20220810           수         17           3          106   

  road_name  multi_linked  connect_code  maximum_speed_limit  ...  \
0       연삼로             0             0                 70.0  ...   
1  일반국도12호선             0             0                 70.0  ...   
2  일반국도16호선             0             0                 60.0  ...   
3  일반국도11호선             0             0                 70.0  ...   
4       번영로             0             0                 70.0  ...   

   height_restricted  road_type  start_node_name  start_latitude  \
0                0.0          0             산지2교      

In [5]:
print(data_info)

                      변수명           변수 설명
0                      id             아이디
1               base_date              날짜
2             day_of_week              요일
3               base_hour             시간대
4             road_in_use          도로사용여부
5              lane_count             차로수
6             road_rating            도로등급
7            multi_linked         중용구간 여부
8            connect_code          연결로 코드
9     maximum_speed_limit          최고속도제한
10      weight_restricted          통과제한하중
11      height_restricted          통과제한높이
12              road_type            도로유형
13         start_latitude        시작지점의 위도
14        start_longitude        시작지점의 경도
15  start_turn_restricted  시작 지점의 회전제한 유무
16           end_latitude        도착지점의 위도
17          end_longitude        도착지점의 경도
18    end_turn_restricted   도작지점의 회전제한 유무
19              road_name             도로명
20        start_node_name           시작지점명
21          end_node_name           도착지점명
22     vehicle_restricted         

In [6]:
train_df.describe()

,base_date,base_hour,lane_count,road_rating,multi_linked,connect_code,maximum_speed_limit,vehicle_restricted,weight_restricted,height_restricted,road_type,start_latitude,start_longitude,end_latitude,end_longitude,target
count,4.701217e+06,4.701217e+06,4.701217e+06,4.701217e+06,4.701217e+06,4.701217e+06,4.701217e+06,4701217.0,4.701217e+06,4701217.0,4.701217e+06,4.701217e+06,4.701217e+06,4.701217e+06,4.701217e+06,4.701217e+06
mean,2.021672e+07,1.192820e+01,1.836651e+00,1.049585e+02,4.762597e-04,2.660218e-01,6.125329e+01,0.0,5.618742e+03,0.0,6.152237e-01,3.338432e+01,1.265217e+02,3.338432e+01,1.265217e+02,4.278844e+01
std,4.555709e+03,6.722092e+00,6.877513e-01,1.840107e+00,2.181818e-02,5.227760e+00,1.213354e+01,0.0,1.395343e+04,0.0,1.211268e+00,1.012015e-01,1.563657e-01,1.011948e-01,1.563519e-01,1.595443e+01
min,2.021090e+07,0.000000e+00,1.000000e+00,1.030000e+02,0.000000e+00,0.000000e+00,3.000000e+01,0.0,0.000000e+00,0.0,0.000000e+00,3.324343e+01,1.261826e+02,3.324343e+01,1.261826e+02,1.000000e+00
25%,2.021110e+07,6.000000e+00,1.000000e+00,1.030000e+02,0.000000e+00,0.000000e+00,5.000000e+01,0.0,0.000000e+00,0.0,0.000000e+00,3.326422e+01,1.264232e+02,3.326422e+01,1.264232e+02,3.000000e+01
50%,2.022013e+07,1.200000e+01,2.000000e+00,1.060000e+02,0.000000e+00,0.000000e+00,6.000000e+01,0.0,0.000000e+00,0.0,0.000000e+00,3.341257e+01,1.265112e+02,3.341257e+01,1.265112e+02,4.300000e+01
75%,2.022050e+07,1.800000e+01,2.000000e+00,1.070000e+02,0.000000e+00,0.000000e+00,7.000000e+01,0.0,0.000000e+00,0.0,0.000000e+00,3.347804e+01,1.265840e+02,3.347804e+01,1.265840e+02,5.400000e+01
max,2.022073e+07,2.300000e+01,3.000000e+00,1.070000e+02,1.000000e+00,1.030000e+02,8.000000e+01,0.0,5.000000e+04,0.0,3.000000e+00,3.355608e+01,1.269309e+02,3.355608e+01,1.269309e+02,1.130000e+02


In [7]:
train_df[train_df['vehicle_restricted'] == 0]['vehicle_restricted']

0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
          ... 
4701212    0.0
4701213    0.0
4701214    0.0
4701215    0.0
4701216    0.0
Name: vehicle_restricted, Length: 4701217, dtype: float64

In [8]:
# vehicle_restricted 컬럼은 전부 0 이므로 필요가 없음

In [9]:
str_col = ['day_of_week','start_turn_restricted','end_turn_restricted','road_name']
for i in str_col:
    le = LabelEncoder()
    le=le.fit(train_df[i])
    train_df[i]=le.transform(train_df[i])
    
    for label in np.unique(test_df[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_df[i]=le.transform(test_df[i])

In [10]:
Y = train_df['target'] 

X = train_df.drop(['id','base_date', 'target', 'start_node_name', 'end_node_name','vehicle_restricted',
                      'end_longitude', 'start_longitude'], axis=1)

X_test = test_df.drop(['id','base_date', 'start_node_name', 'end_node_name','vehicle_restricted',
                 'end_longitude', 'start_longitude'], axis=1)

print(Y.shape)
print(X.shape)
print(X_test.shape)

(4701217,)
(4701217, 15)
(291241, 15)


In [11]:
train_X, val_X, train_y, val_y = train_test_split(X, Y)

In [12]:
# for i in [10000,20000,30000,40000,50000,60000,70000]:
#     model = DecisionTreeRegressor(max_leaf_nodes = i, random_state=1)
#     model.fit(train_X, train_y)
#     predictions = model.predict(val_X)
#     print(mean_absolute_error(val_y, predictions))

In [13]:
LR = lgb.LGBMRegressor(random_state=42).fit(train_X, train_y)

In [14]:
print(mean_absolute_error(LR.predict(val_X), val_y))

5.107865468996465


In [15]:
XGBR = XGBRegressor(random_state=42).fit(train_X, train_y)

In [16]:
print(mean_absolute_error(XGBR.predict(val_X), val_y))

4.03332658512482


In [17]:
RFR = RandomForestRegressor(random_state=42).fit(train_X, train_y)

In [18]:
print(mean_absolute_error(RFR.predict(val_X), val_y))

3.5581862784053557


In [19]:
DTR = DecisionTreeRegressor(random_state=42).fit(train_X, train_y)

In [20]:
print(mean_absolute_error(DTR.predict(val_X),val_y))

3.5589690873838165


In [21]:
lasso = LassoCV(random_state=42).fit(train_X, train_y)

In [22]:
print(mean_absolute_error(lasso.predict(val_X), val_y))

12.287586547871504


In [23]:
ridge = RidgeCV().fit(train_X, train_y)

In [24]:
print(mean_absolute_error(ridge.predict(val_X), val_y))

256.93505415519235


In [ ]:
for i in [100,1000,10000,50000,100000]:
    RFR = RandomForestRegressor(random_state=42, n_estimators=i).fit(train_X, train_y)
    print(mean_absolute_error(RFR.predict(val_X), val_y))

3.5581862784053557
3.5576902690263466


In [ ]:
# predictions = model.predict(test_df[columns])

In [ ]:
# sample_submission['target'] = predictions

In [ ]:
# sample_submission

In [ ]:
# sample_submission.to_csv("../../../data/non-kaggle/Jeju-traffic/submit/Jeju_submit05.csv", index=False)